In [ ]:
import pandas as pd
from pathlib import Path
from os import makedirs

INPUTS_DIR = Path("inputs")
DELINEATION_DIR = INPUTS_DIR / "LOI_delineations" / "LOI_delineations"
PROCESSED_INPUTS_DIR = Path("processed_inputs")
PROCESSED_DELINEATIONS_DIR = PROCESSED_INPUTS_DIR / "delineations"

for dir in [
    INPUTS_DIR,
    DELINEATION_DIR,
    PROCESSED_INPUTS_DIR,
    PROCESSED_DELINEATIONS_DIR,
]:
    makedirs(dir, exist_ok=True)

In [37]:
nc_sites_final = pd.read_csv(INPUTS_DIR / "NC_sites_FINAL_v2_unimpaired_flow.csv")
print(nc_sites_final.columns)

Index(['date', 'flow_cfs', 'unique_ID', 'model_ID_scaled', 'DA_ratio',
       'flow_cfs_modelID', 'Source_File'],
      dtype='object')


In [38]:
nc_sites_final.head()

,date,flow_cfs,unique_ID,model_ID_scaled,DA_ratio,flow_cfs_modelID,Source_File
0,1981-10-01,0.00,NC_CH_1_0862,RWC_3,0.245882,0.00,scaled_flow_NC_CH_1_0862.csv
1,1981-10-02,0.05,NC_CH_1_0862,RWC_3,0.245882,0.19,scaled_flow_NC_CH_1_0862.csv
2,1981-10-03,0.17,NC_CH_1_0862,RWC_3,0.245882,0.69,scaled_flow_NC_CH_1_0862.csv
3,1981-10-04,0.10,NC_CH_1_0862,RWC_3,0.245882,0.42,scaled_flow_NC_CH_1_0862.csv
4,1981-10-05,0.26,NC_CH_1_0862,RWC_3,0.245882,1.06,scaled_flow_NC_CH_1_0862.csv


In [39]:
nc_sites_final.to_csv(Path("processed_inputs/NC_sites_FINAL_v2_unimpaired_flow.csv"))

# Biosites

In [40]:
biosites = pd.read_csv(INPUTS_DIR / "N_Coast_BioSites_Unimpaired_Flow.csv")
print(biosites.columns)

Index(['Unnamed: 0', 'date', 'flow_cfs', 'unique_ID', 'model_ID_scaled',
       'DA_ratio', 'flow_cfs_modelID'],
      dtype='object')


## Notes
Missing `Source_File` column. Extra column **Unnamed: 0**.

## Actions
- Find out from Kris if the Source_File column should be added and filled
- Remove extra column

In [41]:
# Drop "Unnamed: 0" column / keep all other columns
biosites = biosites[
    ["date", "flow_cfs", "unique_ID", "model_ID_scaled", "DA_ratio", "flow_cfs_modelID"]
]

In [42]:
biosites.head()

,date,flow_cfs,unique_ID,model_ID_scaled,DA_ratio,flow_cfs_modelID
0,1981-10-01,0.00,107GWCAPC,RWC_3,0.04077,0.00
1,1981-10-02,0.01,107GWCAPC,RWC_3,0.04077,0.19
2,1981-10-03,0.03,107GWCAPC,RWC_3,0.04077,0.69
3,1981-10-04,0.02,107GWCAPC,RWC_3,0.04077,0.42
4,1981-10-05,0.04,107GWCAPC,RWC_3,0.04077,1.06


## Write processed input file

...to `processed_inputs` directory.

In [43]:
biosites.to_csv(PROCESSED_INPUTS_DIR / "N_Coast_BioSites_Unimpaired_Flow.csv")

# Gages

In [44]:
import geopandas as gp

gages_shp = gp.read_file(DELINEATION_DIR / "All_gages_delineations_combined.shp")
print(gages_shp.columns)
model_site_map = gages_shp[["Modl_ID", "UniquID"]].sort_values(
    by=["Modl_ID", "UniquID"]
)
model_site_map = model_site_map.rename(
    columns={"Modl_ID": "model_ID", "UniquID": "unique_ID"}
)
model_site_map.head()

Index(['Shp_Lng', 'Shap_Ar', 'SUBNUM', 'Subbasn', 'Mdl_bbv', 'Modl_ID',
       'HUC8_Sb', 'Gage_ID', 'Gage_Nm', 'Type', 'Type2', 'Vldtn_N', 'PRMS_Sb',
       'Nsh_S_E', 'R_squrd', 'NRMSE', 'Notes', 'Mdl_bbr', 'UniquID', 'Shp_L_1',
       'StatnCd', 'geometry'],
      dtype='object')


,model_ID,unique_ID
0,ELR_2,None
2,ER_100,Unique100
3,ER_103,Unique103
1,ER_11,Unique011
8,ER_112,Unique112


In [45]:
gages_unimpared = pd.read_csv(INPUTS_DIR / "All_gages_unimpaired_flow.csv")
gages_unimpared = gages_unimpared[["date", "model_ID", "flow_cfs"]].merge(
    model_site_map, how="left"
)
gages_unimpared["date"] = pd.to_datetime(gages_unimpared["date"], format=r"%m/%d/%Y")
print(gages_unimpared.dtypes)
gages_unimpared.head()

date         datetime64[ns]
model_ID             object
flow_cfs            float64
unique_ID            object
dtype: object


,date,model_ID,flow_cfs,unique_ID
0,1982-10-01,ER_8,130.80,Unique008
1,1982-10-02,ER_8,126.11,Unique008
2,1982-10-03,ER_8,122.03,Unique008
3,1982-10-04,ER_8,117.86,Unique008
4,1982-10-05,ER_8,114.07,Unique008


In [ ]:
gages_unimpared.to_csv(PROCESSED_INPUTS_DIR / "All_gages_unimpaired_flow.csv")

# McBain

Shapefile: Rename Site as siteID and 
Flow: Convert date column appropriately

In [64]:
import geopandas as gp
import pandas as pd


mcbain_shp = gp.read_file(DELINEATION_DIR / "McBain_sites_snapped_NAD83_watersheds.shp")
print(mcbain_shp.columns)
mcbain_shp = mcbain_shp.rename(columns={"Site": "siteID"})

mcbain_unimpaired = pd.read_csv(INPUTS_DIR / "McBain_Sites_Unimpaired_Flow.csv")
mcbain_unimpaired["date"] = pd.to_datetime(
    mcbain_unimpaired["date"], format=r"%m/%d/%Y"
)
print(mcbain_unimpaired)

Index(['Shape_Leng', 'Shape_Area', 'Site', 'Notes', 'geometry'], dtype='object')
            date    flow_cfs unique_ID model_ID_scaled  DA_ratio  \
0     1982-10-01  166.823522  EelRiver            ER_1  0.981257   
1     1982-10-02  160.749541  EelRiver            ER_1  0.981257   
2     1982-10-03  155.519440  EelRiver            ER_1  0.981257   
3     1982-10-04  150.122526  EelRiver            ER_1  0.981257   
4     1982-10-05  145.235866  EelRiver            ER_1  0.981257   
...          ...         ...       ...             ...       ...   
72040 2022-03-27    1.114988  Williams           ER_62  0.913924   
72041 2022-03-28    1.297772  Williams           ER_62  0.913924   
72042 2022-03-29    1.581089  Williams           ER_62  0.913924   
72043 2022-03-30    1.361747  Williams           ER_62  0.913924   
72044 2022-03-31    1.297772  Williams           ER_62  0.913924   

       flow_cfs_modelID  
0                170.01  
1                163.82  
2                158.49 

In [48]:
mcbain_unimpaired.to_csv(PROCESSED_INPUTS_DIR / "McBain_Sites_Unimpaired_Flow.csv")

mcbain_shp.to_file(
    PROCESSED_DELINEATIONS_DIR / "McBain_sites_snapped_NAD83_watersheds.shp"
)

c:\Users\AlondraGonzalez\anaconda3\envs\impaired_flows_calculator\lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Value 101176608.120000005 of field Shape_Area of feature 3 not successfully written. Possibly due to too larger number with respect to field width
  ogr_write(
c:\Users\AlondraGonzalez\anaconda3\envs\impaired_flows_calculator\lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Value 9346816508.10000038 of field Shape_Area of feature 7 not successfully written. Possibly due to too larger number with respect to field width
  ogr_write(


# SFE sites high resolution

In [61]:
import geopandas as gp
import pandas as pd
from datetime import datetime, timedelta

highresolution_shp = gp.read_file(
    DELINEATION_DIR / "SFE_sites_highresolution_watersheds.shp"
)

highresolution_unimpaired = pd.read_csv(
    INPUTS_DIR / "SFE_sites_highresolution_Unimpaired_Flow.csv"
)


def excel_serial_to_date(x):
    return datetime(1899, 12, 30) + timedelta(days=x)


highresolution_unimpaired["date"] = highresolution_unimpaired["date"].apply(
    excel_serial_to_date
)
print(highresolution_unimpaired)


highresolution_unimpaired.to_csv(
    PROCESSED_INPUTS_DIR / "SFE_sites_highresolution_Unimpaired_Flow.csv"
)

             date  flow_cfs     unique_ID model_ID_scaled  DA_ratio  \
0      1982-10-01  0.066516  SFE_2017_209          ER_100  0.059925   
1      1982-10-02  0.064119  SFE_2017_209          ER_100  0.059925   
2      1982-10-03  0.061722  SFE_2017_209          ER_100  0.059925   
3      1982-10-04  0.059925  SFE_2017_209          ER_100  0.059925   
4      1982-10-05  0.058127  SFE_2017_209          ER_100  0.059925   
...           ...       ...           ...             ...       ...   
158692 2022-03-27  2.942510   SFE_2018_82          ER_120  0.253446   
158693 2022-03-28  2.988130   SFE_2018_82          ER_120  0.253446   
158694 2022-03-29  3.043888   SFE_2018_82          ER_120  0.253446   
158695 2022-03-30  2.871545   SFE_2018_82          ER_120  0.253446   
158696 2022-03-31  2.782839   SFE_2018_82          ER_120  0.253446   

        flow_cfs_modelID  
0                   1.11  
1                   1.07  
2                   1.03  
3                   1.00  
4           

# SFE sites mainstem

In [72]:
import geopandas as gp
import pandas as pd

mainstem_shp = gp.read_file(DELINEATION_DIR / "SFE_sites_mainstem_watersheds.shp")
print(mainstem_shp.columns)
mainstem_shp = mainstem_shp.rename(columns={"Site": "siteID"})

mainstem_shp.to_file(PROCESSED_DELINEATIONS_DIR / "SFE_sites_mainstem_watersheds.shp")


mainstem_unimpaired = pd.read_csv(INPUTS_DIR / "SFE_sites_mainstem_Unimpaired_Flow.csv")
mainstem_unimpaired["date"] = pd.to_datetime(
    mainstem_unimpaired["date"], format=r"%m/%d/%Y"
)

mainstem_unimpaired.to_csv(
    PROCESSED_INPUTS_DIR / "SFE_sites_mainstem_Unimpaired_Flow.csv"
)


Index(['Shape_Leng', 'Shape_Area', 'Site', 'geometry'], dtype='object')
